# Cupoy官網新聞

## 專案目標

請任選 Cupoy 新聞服務之某一種分類 (如熱門新聞、科技、商業....)，使用你學習過的爬蟲程式，爬取前 500 篇的文章：

https://www.cupoy.com/newsfeed/topstory

In [1]:
#載入套件
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep

In [2]:
#指定網址
Cupoy_url = "https://www.cupoy.com/newsfeed/topicgrp/tech_tw"  

用Selenium打開一個瀏覽器，連結到網站，然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
#使用webdriver.Chrime()建立Google Chrime瀏覽器物件
browser = webdriver.Chrome()
#以get()方法連結到指定網址
browser.get(Cupoy_url)  # 打開瀏覽器並連到Cupoy網頁

SCROLL_PAUSE_TIME = 1

# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#通過左邊控制橫向和縱向滾動條scrollTo(x, y）
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    time.sleep(3) #是Python程序暫停執行3秒
    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print('到達網頁頁面底端')
        break
    print('網頁正在更新中...，請稍後')
    last_height = new_height

網頁正在更新中...，請稍後
網頁正在更新中...，請稍後
到達網頁頁面底端


# 網頁已經更新完畢，所有資料皆已出現在都已出現在網頁。

In [4]:
count = 1
total_count = 500
#建立空的串列(list)
updatetime_list = []
category_list = []
liveness_list = []
title_list = []
content_list = []
href_list = []
while (count <= total_count) :
    html_source = browser.page_source # browser.page_source:網頁原始碼
    soup = BeautifulSoup(html_source, "html5lib")
    
    for artical in soup.find_all('div', class_ = "sc-eEieub sc-iuDHTM ibJqYc"):
        # 文章種類
        category = artical.find('div', class_ = "sc-gacfCG bPSpUf").text
        # 文章標題
        title = artical.find('h6', class_ = 'sc-erNlkL sc-ekulBa hDLssh').text
        # 文章內容
        content = artical.find('p', class_ = 'sc-FQuPU sc-ciodno bvnzOw').text
        # 文章網址
        href = artical.find("a").get('href')
        # 文章更新時間
        updatetime = artical.find('span').text
        # 防止抓取到重複的文章
        if title not in title_list:
            # 將各種資料存放進串列中(list)
            updatetime_list.append(updatetime)
            category_list.append(category)
            title_list.append(title)
            content_list.append(content)
            href_list.append(href)
            # 計算目前總共已經爬取的文章數量
            count += 1
        # 抓取500篇文章就跳出迴圈
        if count == (total_count + 1):
            break
    
    # 將網頁繼續向下滑
    sleep(3) #暫停執行3秒
    browser.execute_script("window.scrollTo(0, 1000000);") # browser.execute_script():滾動到網頁底部

In [5]:
# 關閉瀏覽器
browser.quit();

In [6]:
# 建立 DataFrame
"""
DataFrame 用來處理結構化(Table like)的資料，
有列索引與欄標籤的二維資料集，
可以透過 Dictionary 或是 Array 來建立。
"""
data = pd.DataFrame([updatetime_list,category_list, title_list, content_list, href_list])
# 確認輸出資料形式
print(data) 
#轉置：df.T
data = data.T

                                                 0    \
0                                              1 小時前   
1                                               醫療健康   
2   人造論再次被否！新冠病毒的產生可追溯至 2009 年，SARS 期間或獲得感染能力 | 機器之心   
3  新研究表示，新冠病毒通過重組產生，重組發生的時間有可能出現在 2009 年之前，並獲得 SA...   
4   https://www.jiqizhixin.com/articles/2020-02-22-5   

                                                1    \
0                                             3 小時前   
1                                            AI與大數據   
2            YOLO之父宣佈退出CV界，坦言無法忽視自己工作帶來的負面影響 | 機器之心   
3              沒想到 NeurIPS 的論文接收規則更新，讓 AI 大牛有了新的想法。   
4  https://www.jiqizhixin.com/articles/2020-02-22-3   

                                                 2    \
0                                              4 小時前   
1                                             AI與大數據   
2  新預訓練模型CodeBERT出世，編程語言和自然語言都不在話下，哈工大、中山大學、MSRA出...   
3  可處理雙模態數據的新預訓練模型 CodeBERT，除了自然語言（NL），編程語言（PL）如今...   
4   https://www.jiqizhixin.com/articles/2020-02-22-2

# 整理資料排列方式

In [7]:
# 加入每一行(column)的名稱 
columnName = ["文章更新時間","文章種類","文章標題","文章內容","文章網址"]
# 改欄位名稱
data.columns = columnName
data

,文章更新時間,文章種類,文章標題,文章內容,文章網址
0,1 小時前,醫療健康,人造論再次被否！新冠病毒的產生可追溯至 2009 年，SARS 期間或獲得感染能力 | 機器之心,新研究表示，新冠病毒通過重組產生，重組發生的時間有可能出現在 2009 年之前，並獲得 SA...,https://www.jiqizhixin.com/articles/2020-02-22-5
1,3 小時前,AI與大數據,YOLO之父宣佈退出CV界，坦言無法忽視自己工作帶來的負面影響 | 機器之心,沒想到 NeurIPS 的論文接收規則更新，讓 AI 大牛有了新的想法。,https://www.jiqizhixin.com/articles/2020-02-22-3
2,4 小時前,AI與大數據,新預訓練模型CodeBERT出世，編程語言和自然語言都不在話下，哈工大、中山大學、MSRA出...,可處理雙模態數據的新預訓練模型 CodeBERT，除了自然語言（NL），編程語言（PL）如今...,https://www.jiqizhixin.com/articles/2020-02-22-2
3,4 小時前,AI與大數據,2019 NLP大全：論文、博客、教程、工程進展全梳理（長文預警） | 機器之心,在整個2019年，NLP領域都沉澱了哪些東西？有沒有什麼是你錯過的？放心，有人已經幫你整理好了。,https://www.jiqizhixin.com/articles/2020-02-22
4,4 小時前,AI與大數據,AAAI 2020 | BERT穩嗎？亞馬遜、MIT等提出針對NLP模型的對抗攻擊框架Tex...,不久前，AAAI 2020 在美國紐約召開。爲了向讀者們分享更多的優質內容、促進學術交流，在...,https://www.jiqizhixin.com/articles/2020-02-21-3
...,...,...,...,...,...
495,2020/02/08,醫療健康,到底是誰把新冠病毒帶到武漢？蝙蝠終於不用背鍋，穿山甲或爲中間宿主 | 機器之心,是時候爲蝙蝠正正名了——在自然宿主之外，只有中間宿主才能將病毒與人類連接在一起。,https://www.jiqizhixin.com/articles/2020-02-08
496,2020/02/08,理財,聯發科 2019 年 EPS 達 14.69 元，瞄準 AI、5G、Wi-Fi 6 等新技術...,IC 設計大廠聯發科 7 日召開線上法說會，由執行長蔡力行及財務長兼發言人顧大為主持。會中發...,https://technews.tw/2020/02/07/mediatek-2019-f...
497,2020/02/07,Fintech金融科技,如何關閉自動播放？Netflix 新增關閉選項 - INSIDE,Netflix 的自動播放讓很多觀眾養成一次追完整季影集的習慣，但如果你就是想一次看一集，現...,https://www.inside.com.tw/article/18843-How-to...
498,2020/02/07,Fintech金融科技,武漢肺炎疫情衝擊 Apple 中國供應鏈 - INSIDE,華爾街日報認為蘋果可能在這波疫情下大受影響，雖然 iPhone 部分在印度生產，但大多是為了...,https://www.inside.com.tw/article/18847-wuhan-...


In [8]:
# 回傳列數與欄數 
print("資料行數: ",data.shape[0]) 
print("資料列數: ",data.shape[1]) 
print("----------------------------------------------------------------------------------------------------") 
# 回傳欄位名稱
print(data.columns)

資料行數:  500
資料列數:  5
----------------------------------------------------------------------------------------------------
Index(['文章更新時間', '文章種類', '文章標題', '文章內容', '文章網址'], dtype='object')


# 儲存CSV

將爬取到的500筆新聞資料儲存一個.CSV檔案。

In [9]:
# 儲存成 .xlsx
data.to_csv("Cupoy_news.csv", encoding="utf-8")

# 類別分類

In [10]:
import pandas as pd
category_count = {}

for category in category_list:
    #如果該類別不存在，則計算第一次
    if category not in category_count:
        category_count[category] = 1
    #類別存在，則計算數量    
    else:
        category_count[category] += 1

print('類別:', category_count)
print('類別最多的為:', max(category_count, key = category_count.get))
# 類別名稱
labels = list(category_count.keys())
# 類別數量
values = list(category_count.values())
category_count_data = pd.DataFrame(values,index=labels,columns = ["數量"])
print(category_count_data)
#輸出計算結果
print("總共類別數: ", category_count_data.shape[0])
print("==========================================================================================================")
# 回傳欄位名稱
print(category_count_data.columns)      

類別: {'醫療健康': 6, 'AI與大數據': 184, '電子商務': 11, '創業': 5, '資訊安全': 23, '無人駕駛': 4, 'Google專區': 10, '區塊鏈技術': 27, '遊戲情報': 13, '3C產品': 13, '蘋果專區': 18, 'APP應用程式': 24, '手機平板': 13, '理財': 4, '相機': 5, '趨勢與策略': 10, '虛擬實境': 6, '社交媒體': 11, '職涯': 7, 'Fintech金融科技': 85, '筆電': 4, '美妝保養': 1, '物聯網': 4, '網紅經濟與自媒體': 1, '財經': 3, '美食': 2, '籃球': 2, '旅遊': 1, '育兒': 1, '行銷': 2}
類別最多的為: AI與大數據
              數量
醫療健康           6
AI與大數據       184
電子商務          11
創業             5
資訊安全          23
無人駕駛           4
Google專區      10
區塊鏈技術         27
遊戲情報          13
3C產品          13
蘋果專區          18
APP應用程式       24
手機平板          13
理財             4
相機             5
趨勢與策略         10
虛擬實境           6
社交媒體          11
職涯             7
Fintech金融科技   85
筆電             4
美妝保養           1
物聯網            4
網紅經濟與自媒體       1
財經             3
美食             2
籃球             2
旅遊             1
育兒             1
行銷             2
總共類別數:  30
Index(['數量'], dtype='object')


# 數據資料視覺化

In [40]:
# 載入套件
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties# import字型管理套件

myfont = FontProperties(fname = r"./NotoSansCJK-Black.ttc")

#使用rcParams 參數指定字體
# 中文亂碼的處理，rcParams也可以用於設定圖的解析度，大小等資訊
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False
# 設定圓餅圖大小
plt.figure(figsize=(200,100))
#explode = (0,0,0.05,0)
l_text,p_text = plt.pie(values,
                        labels = labels, # 分類的標記
                        labeldistance = 1.1,
                        pctdistance = 0.85, # 數值與圓餅圖的圓心距離
                        radius = 0.9,       # 圓餅圖的半徑，預設是1
                        center = (0,0),     # 圓餅圖的圓心座標
                        textprops = {"fontsize" : 120}, # 文字大小
                        shadow=False)       # 是否使用陰影

#設定劃出正元餅圖
plt.axis("equal")
#設定橫縱座標的名稱以及對應字型格式
font = {'size': 120}
# 設定legnd的位置
plt.legend(loc = "center right",prop=font)
plt.show()

plt.savefig("Pie chart of Cupoy_news") # 儲存圖檔

<Figure size 432x288 with 0 Axes>

In [ ]:
#載入套件
from wordcloud import WordCloud, ImageColorGenerator
import jieba
import numpy as np
from collections import Counter
# 準備標題語料
titleCorpus = ''
for title in title_list:
    titleCorpus += title 

# 準備內容語料 
contentCorpus = ''
for content in content_list:
    contentCorpus += content  

# 設定停用詞
with open('D:\\Jupyter\\1st-PyCrawlerMarathon\stopWords.txt', 'r', encoding="UTF-8") as f:
    stops = f.read().split('\n')

# 對titleCorpus進行斷詞
term_titleCorpus = []
for word in cut(titleCorpus):
    if word not in stops:
        term_titleCorpus.append(word)
titleCount = dict(Counter(term_titleCorpus))

# 對contentCorpus進行斷詞
term_contentCorpus = []
for word in cut(contentCorpus):
    if word not in stops:
        term_contentCorpus.append(word)
contentCount = dict(Counter(term_contentCorpus))

In [ ]:
# 字體路徑設定
font = r'msjh.ttc'

# 標題文字雲繪圖
title_wordcloud = WordCloud(background_color="white",font_path = font, collocations=False, width=1200, height=1200, margin=2)  
title_wordcloud.generate_from_frequencies(frequencies = titleCount)
plt.figure(figsize=(5,5))
plt.title('新聞標題文字雲', fontsize = '18')
plt.imshow(title_wordcloud,interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()

# 內容文字雲繪圖
content_wordcloud = WordCloud(background_color="white",font_path = font, collocations=False, width=1200, height=1200, margin=2)  
content_wordcloud.generate_from_frequencies(frequencies = contentCount)
plt.figure(figsize=(5,5))
plt.title('新聞簡介文字雲', fontsize = '18')
plt.imshow(content_wordcloud,interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()